In [ ]:
import os
from collections import namedtuple
from pprint import pprint
import pandas as pd
from tqdm import tqdm

In [5]:
traffic = pd.read_csv('traces/traffic_flows.csv')
drop_cols = traffic.columns[1]
traffic = traffic.drop(columns=['lpid'])

In [6]:
path = os.getcwd() + "/topologies/zte_topo/"
switch_config_paths = [path + conf for conf in os.listdir(path)]

In [7]:
'''
    namedtuples defining a switch
'''
port = namedtuple("port", ["name","bandwidth", "ip", "mask"])
port_from_list = lambda x: port(
    name        = x[0],
    bandwidth   = int(x[1]),
    ip          = x[2],
    mask        = int(x[3])
)

route = namedtuple("route", ["src", "mask", "dest", "dest_id", "port"])
route_from_list = lambda x: route(
    src     = x[0], 
    mask    = int(x[1]), 
    dest    = x[2],
    dest_id = int(x[3]), 
    port    = x[4]
)

topo = namedtuple("topo", ["dest_id", "src_port", "dest_port"])
topo_from_list = lambda x: topo(
    dest_id    = int(x[0]), 
    src_port   = x[1], 
    dest_port  = x[2])

switch = namedtuple("switch", ["id", "type", "ports", "routes", "topos"])

# formatted print for switches
show_switch = lambda x: pprint(x._asdict())

In [8]:
def switch_from_config(conf):
    '''
        Creates switch namedtuples from switch config files
    '''
    with open(conf, 'r') as f:
        switch_data_lines = [line.strip().split(',') for line in f.readlines()]

    ports, routes, topos = [], [], []
    for switch_data in switch_data_lines:    
        match switch_data.pop(0):
            case "info":
                id, switch_type = int(switch_data[0]), switch_data[1]
            case "port":
                ports.append(port_from_list(switch_data))
            case "route":
                routes.append(route_from_list(switch_data))
            case "topo":
                topos.append(topo_from_list(switch_data))

    return switch(
        id      = id, 
        type    = switch_type, 
        ports   = ports, 
        routes  = routes, 
        topos   = topos
    )

In [ ]:
switches = {}
for path in tqdm(switch_config_paths):
    sw = switch_from_config(path)
    switches[sw.id] = sw

  0%|          | 0/9243 [00:00<?, ?it/s]

100%|██████████| 9243/9243 [00:29<00:00, 313.10it/s]


In [35]:
def print_local_ports(ports, dest):
    print('local ports:')
    print(f"\t {"LOCAL PORT IP":18} {"DEST PORT IP":18}  | IS DESTINATION?")
    for idx,p in enumerate(ports):
        print(f"\t {str(p):18}  {str(dest):18} | {p == dest}")

def print_subnets(subnets, dest):
    print('subnets:')
    print(f"\t {"SUB NET":18} {"DEST PORT IP":18}  | DEST PORT IN SUBNET?")
    for idx,(route,next_id) in enumerate(subnets):
        print(f"\t {str(route):18}  {str(dest):18} | {dest in route}")

def print_candidate_next_hops(cnh):
    print("candidate next hops:")
    print(f"\t {"SUBNETWORK":18} {"SWITCH_ID":18}")
    for next in cnh:
        print(f"\t {str(next[0]):18} {str(next[2]):18}")


In [51]:
import ipaddress

get_switch_port_ips = lambda switch : [ipaddress.ip_address(port.ip) for port in switch.ports]
network_from_route  = lambda route  : ipaddress.ip_network(f"{route.src}/{route.mask}")

def calculate_flow_path(flow):
    source = flow['switch_id']
    dest   = ipaddress.ip_address(flow['dest_switch_ip'])
    print(f"flow from switch {source} to IP {dest} ...")

    current_switch = switches[source]
    print("STARTING AT:",current_switch.id)

    flow_path = []

    while True:
        print("="*100)
        # add switch to flow path
        flow_path.append(current_switch)

        # get local port IPs
        ports = get_switch_port_ips(current_switch)
        print_local_ports(ports, dest)
        
        # check if we reached dest
        if dest in ports:
            print(f"FINISHED! Found a switch with port {dest} -> switch {current_switch.id}")
            break
        
        # get a list of all subnets accessible from current_switch
        subnets = [(network_from_route(route), route.dest_id) for route in current_switch.routes]
        print_subnets(subnets, dest)

        # get a list of (subnet, mask, dest_id) for all subnets that contain the dest ip
        candidate_next_hops = [(route, route.prefixlen, dest_id) for route, dest_id in subnets if dest in route]
        print_candidate_next_hops(candidate_next_hops)
    
        # get the subnet with the largest netmask that contains the dest ip
        largest_net_mask_subnet = max(candidate_next_hops, key= lambda x:x[1])
        next_hop_id = largest_net_mask_subnet[2]

        print("HOPPING TO:", next_hop_id)
        current_switch = switches[next_hop_id]

    return flow_path

In [52]:
# for idx,flow in enumerate(traffic.itertuples()):
#     flow = flow._asdict()
#     try:
#         trace_flow(flow)
#     except:
#         print(f"flow {idx} failed...")

calculate_flow_path(traffic.iloc[4])

flow from switch 299 to IP 8.25.210.211 ...
STARTING AT: 299
local ports:
	 LOCAL PORT IP      DEST PORT IP        | IS DESTINATION?
	 8.25.210.106        8.25.210.211       | False
	 8.25.212.248        8.25.210.211       | False
	 8.25.212.247        8.25.210.211       | False
subnets:
	 SUB NET            DEST PORT IP        | DEST PORT IN SUBNET?
	 8.25.212.246/31     8.25.210.211       | False
	 8.25.212.247/32     8.25.210.211       | False
	 8.25.212.248/31     8.25.210.211       | False
	 8.25.212.248/32     8.25.210.211       | False
	 8.25.210.106/32     8.25.210.211       | False
	 8.25.210.211/32     8.25.210.211       | True
	 8.25.194.250/31     8.25.210.211       | False
candidate next hops:
	 SUBNETWORK         SWITCH_ID         
	 8.25.210.211/32    3987              
HOPPING TO: 3987
local ports:
	 LOCAL PORT IP      DEST PORT IP        | IS DESTINATION?
	 8.25.210.1          8.25.210.211       | False
	 8.0.69.6            8.25.210.211       | False
	 8.25.211.4     

ValueError: max() iterable argument is empty